This notebook contains work that I had no time to finish. Good comments are missing.

In [ ]:
# Imports
import json
import os
import numpy as np
import time
from random import random, randint, choice

This is a class to handle ARC dataset

In [ ]:
def get_dataset():
    dataset = {
        "train": {},
        "test": {},
        "evaluation": {}
    }
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            beg = dirname.split('/')[-1][:3]
            if beg == "abs":
                continue
            print(os.path.join(dirname, filename))
            with open(os.path.join(dirname, filename)) as json_file:
                data = json.load(json_file)
                key = filename[:-5]
                if beg == "eva":
                    dataset["evaluation"][key] = data
                if beg == 'tes':
                    dataset["test"][key] = data
                if beg == 'tra':
                    dataset["train"][key] = data
    return dataset



class Task:
    '''
        Handle a Task of the ARC Dataset
    '''
    def __init__(self, task_name, task_data):
        self.name = task_name
        self.data = task_data

    def examples(self):
        return map(lambda x: (x['input'], x['output']), self.data['train'])

    def test(self):
        return map(lambda x: x['input'], self.data['test'])

    def get_solutions(self):
        return map(lambda x: x['output'], self.data['test'])

class ARC:
    '''
        Handle the ARC Dataset
    '''
    def __init__(self):
        self.dataset = get_dataset()

    def tasks(self, task_type = 'all'):
        '''
            Iterate over tasks of task_type type

            Params:
                task_type: 'train', 'test', 'eval', 'all'

            Return:
            Iterator
        '''
        for t_type, tasks in self.dataset.items():
            if task_type != 'all' and task_type != t_type:
                continue
            for t_name, task in tasks.items():
                yield Task(t_name, task)

In [ ]:
arc = ARC()

In [ ]:
import numpy as np

def get_shape_maps(img):
    shape_matrix = np.zeros_like(img, dtype=np.uint8)
    shape_matrix_s = np.zeros_like(img, dtype=np.uint8)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            pix = int(img[i, j])
            if pix == 0:
                continue
            neighboors = []
            neighboors_s = []
            if j > 0:
                neighboors.append(shape_matrix[i, j - 1])
                neighboors_s.append(shape_matrix_s[i, j - 1])
            if i > 0:
                if j > 0:
                    neighboors.append(shape_matrix[i - 1, j - 1])
                neighboors.append(shape_matrix[i - 1, j])
                neighboors_s.append(shape_matrix_s[i - 1, j])
                if j < img.shape[1] - 1:
                    neighboors.append(shape_matrix[i - 1, j + 1])
            neighboors = list(set(filter(lambda x: x > 0, neighboors)))
            neighboors_s = list(set(filter(lambda x: x > 0, neighboors_s)))
            if len(neighboors) == 0:
                shape_n = shape_matrix.max() + 1
                shape_matrix[i, j] = shape_n
            else:
                iterator = iter(neighboors)
                ref = next(iterator)
                shape_matrix[i, j] = ref
                for n in iterator:
                    shape_matrix[shape_matrix==n] = ref

            if len(neighboors_s) == 0:
                shape_n = shape_matrix_s.max() + 1
                shape_matrix_s[i, j] = shape_n
            else:
                iterator = iter(neighboors_s)
                ref = next(iterator)
                shape_matrix_s[i, j] = ref
                for n in iterator:
                    shape_matrix_s[shape_matrix_s==n] = ref
    return shape_matrix, shape_matrix_s

def truncate(shape):
    ligne_s_l = shape.sum(axis = 1) > 0
    ligne_s_c = shape.sum(axis = 0) > 0
    l_i = np.argwhere(ligne_s_l)
    c_i = np.argwhere(ligne_s_c)
    shape = shape[l_i.min() : l_i.max() + 1]
    shape = shape[:, c_i.min() : c_i.max() + 1]
    return shape, (np.argmax(ligne_s_l), np.argmax(ligne_s_c))

class Image:
    def __init__(self, pixels):
        self.pixels = np.array(pixels)
        self.find_shapes()
        self.calculate_features()

    def find_shapes(self):
        self.shapes = []
        for i in range(10):
            img = (self.pixels == i).astype(np.uint8)
            if img.sum() == 0:
                continue
            t_shape, offset = truncate(img)
            self.shapes.append({
                "blob": t_shape,
                "color": i,
                "offset": offset
            })
            smaps = get_shape_maps(img)
            for smap in smaps:
                for c in range(1, smap.max()+1):
                    shape = (smap == c).astype(np.uint8)
                    if shape.sum() == 0:
                        continue
                    t_shape, offset = truncate(shape)
                    self.shapes.append({
                        "blob": t_shape,
                        "color": i,
                        "offset": offset
                    })
                    
    def calculate_features(self):
        self.features = {
            "height": self.pixels.shape[0],
            "width": self.pixels.shape[1],
            "n_shapes": len(self.shapes),
        }
        colors = {}
        c_colors = 0
        for i in range(10):
            colors[i] = (self.pixels == i).sum()
            self.features['color_'+str(i)+'_count'] = colors[i]
            if colors[i] > 0:
                c_colors += 1
        self.features['number_colors'] = c_colors
    
    @staticmethod
    def get_features_len():
        return len(Image([[1]]).features)
    
    @staticmethod
    def get_features_list():
        return list(Image([[1]]).features.keys())

    def __str__(self):
        return str(self.pixels)

In [ ]:
# Let's do some statistics and measure performance of our algorithm


def time_processing(ds='train'):
    n_y, n_y_in_x, n_x, n_x_in_y = 0, 0, 0, 0
    start_t = time.time()
    n_shape = []
    for task in arc.tasks(ds):
        #print(task.name)
        for X, y in task.examples():
            X = Image(X)
            y = Image(y)
            l_x = len(X.shapes)
            l_y = len(y.shapes)
            n_shape.append(l_x)
            n_shape.append(l_y)
            n_x += l_x
            n_y += l_y

        for X, y in zip(task.test(), task.get_solutions()):
            X = Image(X)
            y = Image(y)
            n_shape.append(len(X.shapes))
            n_shape.append(len(y.shapes))

    n_shape = np.array(n_shape)
    duration = time.time() - start_t
    print(round(10000*duration)/10000,'s for processing',len(n_shape),'images (avg',round(10000*duration/len(n_shape))/10000,'s/img)')
    print('Mean', n_shape.mean())
    print('Std', n_shape.std())
    print('Min', n_shape.min())
    print('Max', n_shape.max())
    
    
time_processing()
#time_processing('evaluation')

Now we need a strategy to find the shape of y given the image X

In [ ]:
def transpose(X):
    return [X.T]

def rotate(X):
    if X.shape[0] * X.shape[1] == 1:
        return [X]
    responses = []
    for i in range(3):
        rot = np.rot90(X, i+1)
        responses.append(rot)
    return responses

def equal(X):
    return [X]

transformations = [
    equal,
    transpose,
    rotate
]

class Transformation:
    @staticmethod
    def compare(X, y):
        matches = []
        shapes = []
        t_names = []
        for f in transformations:
            res = f(X)
            shapes += res
            t_names += list(map(lambda x: f.__name__ + '_' + str(x), range(len(res))))
        for i, s in enumerate(shapes):
            if s.shape == y.shape and (s == y).all():
                if s.shape[0] * s.shape[1] == 1:
                    continue
                f_name = t_names[i]
                if f_name == 'equal_0':
                    return ['equal']
                matches.append(f_name)
        return matches
        
        

In [ ]:
class Strategy:
    def __init__(self):
        pass
    
    def observe(self, X, y):
        similar = []
        for i, s_x in enumerate(X.shapes):
            for j, s_y in enumerate(y.shapes):
                matches = Transformation.compare(s_x['blob'], s_y['blob'])
                for m in matches:
                    similar.append({
                        "from": i,
                        "to": j,
                        "with": m
                    })
        print('X')
        print(X)
        print('y')
        print(y)
        print('Found',len(similar),'matches on',len(X.shapes),'x',len(y.shapes),'shapes:')
        print(similar)
        for i, s_x in enumerate(X.shapes):
            print('X['+str(i)+']')
            print('Blob:')
            print(s_x['blob'])
            print('Color:',s_x['color'])
            print('Offset:',s_x['offset'])
        for i, s_y in enumerate(y.shapes):
            print('Y['+str(i)+']')
            print('Blob:')
            print(s_y['blob'])
            print('Color:',s_y['color'])
            print('Offset:',s_y['offset'])


In [ ]:
for task in arc.tasks('train'):
    if random() < 0.95:
        continue
    #print(task.name)
    #strat = Strategy()
    
    for X, y in task.examples():
        X = Image(X)
        y = Image(y)
        
        #strat.observe(X, y)
        break
    break
            

In [ ]:
class VarFinder:
    def __init__(self):
        self.is_constant = True
        self.constant_v = None
        self.delta = None
        self.scale = None
        self.delta_selector = None
        self.scale_selector = None
        self.firstTime= True
        self.features = None
        
    def get_features_vector(self, features):
        return np.array(list(map(lambda x: features[x], self.features)))
        
    def observe(self, features, v):
        if self.firstTime:
            self.constant_v = v
            self.features = sorted(features.keys())
            self.delta_selector = np.ones(len(features)).astype(np.bool)
            self.scale_selector = np.ones(len(features)).astype(np.bool)
        else:
            if v != self.constant_v:
                self.is_constant = False
        if self.delta_selector.sum() == 0 and self.scale_selector.sum() == 0:
            return
        f = self.get_features_vector(features)
        zeros = f == 0
        f[zeros] = 1
        if v != 0:
            self.delta_selector = np.logical_and(np.logical_not(zeros), self.delta_selector)
            self.scale_selector = np.logical_and(np.logical_not(zeros), self.scale_selector)
        a = v / f
        f[zeros] = 0
        b = v - f
        integers = (a - np.round(a, decimals=1)) == 0.
        self.scale_selector = np.logical_and(integers, self.scale_selector)
        if self.firstTime:
            self.delta = b
            self.scale = a
            self.firstTime = False
        else:
            self.delta_selector = np.logical_and(self.delta_selector, self.delta == b)
            self.scale_selector = np.logical_and(self.scale_selector, self.scale == a)
            self.delta = self.delta_selector * b
            self.scale = self.scale_selector * a
        
            
    def predict(self, features, no_const=False):
        if self.delta_selector.sum() == 0 and self.scale_selector.sum() == 0:
            return None
        f = self.get_features_vector(features)
        p1 = f[self.delta_selector] + self.delta[self.delta_selector]
        p2 = f[self.scale_selector] * self.scale[self.scale_selector]
        votes = {}
        p = p1.tolist() + p2.tolist()
        for v in p:
            votes[v] = votes.get(v, 0) + 1
        computed = max(votes.items(), key=lambda x: x[1])[0]
        if not no_const and self.is_constant:
            if self.constant_v != computed and votes[computed] > 2:
                return computed
            return self.constant_v
        return computed
        
    def __str__(self):
        ret = "Correlations:\n"
        ret += "is_constant : {}\n".format(self.is_constant)
        ret += "constant_v : {}\n".format(self.constant_v)
        ret += "delta : {}\n".format(self.delta)
        ret += "scale : {}\n".format(self.scale)
        ret += "delta_selector : {}\n".format(self.delta_selector)
        ret += "scale_selector : {}\n".format(self.scale_selector)
        ret += "firstTime : {}\n".format(self.firstTime)
        ret += "features : {}\n".format(self.features)
        return ret
            

In [ ]:
total, good, prononced = 0, 0, 0
similaritys = []

for task in arc.tasks('train'):
    #print(task.name)
    vf = {
        "height": VarFinder(),
        "width": VarFinder()
    }
    for X, y in task.examples():
        X = Image(X)
        y = Image(y)
        for k, v in vf.items():
            v.observe(X.features, y.features[k])
    for X, y in zip(task.test(), task.get_solutions()):
        X = Image(X)
        y = Image(y)
        similarity = 0
        if X.pixels.shape == y.pixels.shape:
            similarity = (X.pixels == y.pixels).sum() / (X.pixels.shape[0] * X.pixels.shape[1])
        similaritys.append(similarity)
        prediction = {}
        for k, v in vf.items():
            prediction[k] = v.predict(X.features, similarity > 0.75)
        predicted_shape = (prediction['height'], prediction['width'])        
        total += 1
        if predicted_shape[0] != None and predicted_shape[1] != None:
            prononced += 1
            if predicted_shape == y.pixels.shape:
                good += 1
            else:
                print(similarity)
                print("Error at",task.name,"Predicted",predicted_shape,"Instead of",y.pixels.shape)
    
similaritys = np.array(similaritys)
print("Total:", total, "Prononced:",prononced, "Good",good)
print("Total score:",round(1000*good/total)/1000)
print("Prononced score:",round(1000*good/prononced)/1000)

print('Mean:',similaritys.mean())
print('Std:',similaritys.std())
print('Max:',similaritys.max())
print('Min:',similaritys.min())

Ok now we have a good way for predicting shape. Now let's construct drawers

In [ ]:
def get_min(X, f):
    return min(X, key=lambda x: x[f])
    
def get_max(X, f):
    return max(X, key=lambda x: x[f])

sel_func = [
    get_min,
    get_max
]

img_features = [
    'height',
    'width',
    'blob_count',
    'color_count'
]

def select(X, n):
    f_index = n // len(img_features)
    a_index = n%len(img_features)
    
    imgs = []
    for s in X.shapes:
        blob_count, color_count = 0, 0
        for s2 in X.shapes:
            if s['blob'].shape == s2['blob'].shape and (s['blob'] == s2['blob']).all():
                blob_count += 1
            if s['color'] == s2['color']:
                color_count += 1
        imgs.append({
            "blob": s['blob'],
            "height": s['blob'].shape[0],
            "width": s['blob'].shape[1],
            "color": s['color'],
            "blob_count": blob_count,
            "color_count": color_count
        })

    return sel_func[f_index](imgs, img_features[a_index])

class Selector:
    @staticmethod
    def select_shape(X, n):
        if n == Selector.get_shape_sel_max() - 1:
            return np.array([[1]])
        shape = select(X, n)
        return shape["blob"]
        
    @staticmethod
    def select_var(X, n):
        if n == Selector.get_var_sel_max() - 1:
            return 0
        f_i = n // (len(sel_func) * len(img_features))
        shape = select(X, n%(len(sel_func) * len(img_features)))
        return shape[img_features[f_i]]
    
    @staticmethod
    def get_shape_sel_max():
        return 1 + len(sel_func) * len(img_features)
    @staticmethod
    def get_var_sel_max():
        return 1 + len(sel_func) * len(img_features) * len(img_features)
    
    def shape_selection_name(idx):
        return 

In [ ]:
class Canvas:
    def __init__(self, shape):
        self.shape = shape
        self.pixels = None
    
    def initialize(self, X):
        if self.shape != X.shape:
            print('WARNING ! you try to initialize a canvas with a shape he is not supposed to have')
        self.pixels = np.copy(X)
        
    def make(self, blob, color, pos):
        if self.pixels is None:
            print("You must initialize the canvas before yo draw on it")
            return
        max_height = self.pixels.shape[0] - pos[0]
        max_width = self.pixels.shape[1] - pos[1]
        if max_height <= 0 or max_width <= 0:
            return False
        truncated_blob = blob[:min(max_height, blob.shape[0]), :min(max_width, blob.shape[1])]
        self.pixels[pos[0] : pos[0] + truncated_blob.shape[0], pos[1] : pos[1] + truncated_blob.shape[1]] = truncated_blob
        
        return True
        
    def get(self):
        return self.pixels
    
    def __str__(self):
        return str(self.pixels)

In [ ]:
GEOMETRIC_PARAM = 0.95
NB_COLORS = 10
# Hyper params
KEEP_PARENT_PROB = 0.7
NB_MAX_OP = 5
KEEP_PROB = 0.9

genome_shape = [
    Selector.get_shape_sel_max(),
    Selector.get_var_sel_max(),
    Selector.get_var_sel_max(),
    Selector.get_var_sel_max()
]

def get_random_genome():
    genome = []
    for m in genome_shape:
        genome.append(randint(0, m - 1))
    return genome

def new_genome():
    genome = []
    p = KEEP_PROB
    for _ in range(NB_MAX_OP):
        if random() < p:
            genome.append(get_random_genome())
            p *= KEEP_PROB
        else:
            break
    return randint(0, 1), genome

class Drawer:
    def __init__(self, genome=None):
        if genome is None:
            genome = new_genome()
        self.genome = genome
        self.fitness = None
        
    def draw(self, X, shape):
        starter, program = self.genome
        canvas = Canvas(shape)
        if X.pixels.shape == shape and starter == 0:
            canvas.initialize(X.pixels)
        else:
            self.genome = (1, program)
            canvas.initialize(np.zeros((shape)))
        
        to_del = []
        for line in program:
            shape_sel = line[0]
            color = line[1]
            offset_x = line[2]
            offset_y = line[3]
            shape = Selector.select_shape(X, shape_sel)
            c_color = Selector.select_var(X, color)
            if c_color not in range(10):
                color = randint(0, 9)
            line[1] = color
            offset = Selector.select_var(X, offset_x), Selector.select_var(X, offset_y)
            if not canvas.make(shape, color, offset):
                to_del.append(line)
                
        for d in to_del:
            program.remove(d)
                
        return canvas.get()
    
    def get_program(self):
        prog = []
        for l in self.genome[1]:
            nl = []
            nl.append(Selector.shape_selection_name(l[0]))
            nl.append(Selector.var_sel_name(l[1]))
            nl.append(Selector.var_sel_name(l[2]))
            nl.append(Selector.var_sel_name(l[3]))
            prog.append(nl)
        return prog
    
    def make_bebe(self, parent):
        genome = []
        p = KEEP_PROB
        for i in range(NB_MAX_OP):
            if random() < p:
                if random() < KEEP_PARENT_PROB:
                    poss = []
                    if i < len(self.genome[1]):
                        poss.append(self.genome[1][i])
                    if i < len(parent.genome[1]):
                        poss.append(parent.genome[1][i])
                    if len(poss) > 0:
                        genome.append(choice(poss))
                    else:
                        genome.append(get_random_genome())
                else:
                    genome.append(get_random_genome())
                p *= KEEP_PROB
            else:
                break
        n_one = Drawer((randint(0, 1) if self.genome[0] != parent.genome[0] else self.genome[0], genome))
        return n_one
    
    def mutate(self):
        if len(self.genome[1]) > 0 and random() < DELETE_PROB * len(self.genome[1]):
            del self.genome[1][randint(0, len(self.genome[1]) - 1)]
        
    def __str__(self):
        return str(self.genome)

In [ ]:
# Hyperparams
POPULATION_SIZE = 500
MAX_EPOCHS = 5

class DrawerPool:
    def __init__(self):
        self.population = []
        self.solution = []
        self.old_pop = []
        for _ in range(POPULATION_SIZE):
            self.population.append(Drawer())

    def train(self, Xs, ys):
        self.old_pop = []
        for i in range(MAX_EPOCHS):
            fitnesses = []
            for n, drawer in enumerate(self.population):
                if n%100 == 0:
                    print(n, '/', len(self.population))
                scores = []
                for X, y in zip(Xs, ys):
                    pred = drawer.draw(X, y.pixels.shape)
                    scores.append((pred == y.pixels).sum() / (y.pixels.shape[0] * y.pixels.shape[1]))
                scores = np.array(scores)
                if scores.mean() == 1.0:
                    self.solution.append(drawer)
                else:
                    fitness = scores.mean()# / scores.std()
                    drawer.fitness = fitness
                    fitnesses.append(fitness)
                
            self.old_pop = self.population
            fitness_sum = sum(fitnesses)
            print(max(self.population, key=lambda x: x.fitness).draw(Xs[0], ys[0].pixels.shape))
            self.population = []
            for _ in range(POPULATION_SIZE):
                p1 = self.pick_one(self.old_pop, fitness_sum)
                p2 = self.pick_one(self.old_pop, fitness_sum)
                self.population.append(p1.make_bebe(p2))
                
            fitnesses = np.array(fitnesses)
            print('EPOCH',i)
            print('AVG fitness :',fitnesses.mean())
            print('STD fitness :',fitnesses.std())
            print('Max fitness :',fitnesses.max())
            print('Min fitness :',fitnesses.min())
            if len(self.solution) >= 3:
                print('GOT IT')
                break
            
    def pick_one(self, population, f_sum):
        r = random()
        s = 0
        pop = iter(population)
        while s < r:
            try:
                c = next(pop)
            except:
                break
            s += c.fitness / f_sum
        return c

    def predict(self, X, o_shape):
        responses = []
        for drawer in self.solution + sorted(self.old_pop, key=lambda x: x.fitness, reverse=True):
            if len(responses) >= 3:
                break
            responses.append(drawer.draw(X, o_shape))
            print(drawer.get_program())
        return responses
    

In [ ]:
total, good, prononced = 0, 0, 0

for task in arc.tasks('train'):
    if random() < 0.99:
        continue
    print(task.name)
    dp = DrawerPool()
    vf = {
        "height": VarFinder(),
        "width": VarFinder()
    }
    Xs, ys = [], []
    for X, y in task.examples():
        X = Image(X)
        y = Image(y)
        f_x = X.features
        f_y = y.features
        Xs.append(X)
        ys.append(y)
        
        for k, v in vf.items():
            v.observe(f_x, f_y[k])
            
    dp.train(Xs, ys)
    
    for X, y in zip(task.test(), task.get_solutions()):
        X = Image(X)
        y = Image(y)
        similarity = 0
        if X.pixels.shape == y.pixels.shape:
            similarity = (X.pixels == y.pixels).sum() / (X.pixels.shape[0] * X.pixels.shape[1])
        prediction = {}
        for k, v in vf.items():
            prediction[k] = v.predict(X.features, similarity > 0.75)
        predicted_shape = (prediction['height'], prediction['width'])        
        predictions = dp.predict(X, predicted_shape)
        print(json.dumps([
            predictions[0].tolist(),
            y.pixels.tolist()
        ]))
        print('TODO: Evaluation method')
    break
